In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from gl_functions import *
from spot_functions import *
from constants import timeout_sec, upload_filepath

In [3]:
#### CONSTANTS ####
all_wp_move_look = [(10, [0, 0, np.pi/4], [2, 0, 2.5]),
                    (18, [0, 0, np.pi/2], [1, 0, 5.0]),
                    (22, [0, 0, np.pi/3], [2, 0, 4.0]),
                    (0 , [0, 0, np.pi/4], [3, 0 ,3.0])]

In [4]:
sdk, robot, id_client, \
robot_state_client, command_client, \
image_client, graph_nav_client, \
world_object_client, manipulation_api_client = init_robot('192.168.50.3')

In [5]:
lease_client, lease, lease_keep_alive = get_lease(robot)

In [6]:
gl = gl_init()
light_det = gl.list_detectors().results[0]

In [182]:
get_batt_info(robot_state_client)

Battery at: 38.0 %. Robot has 2147 seconds left


(38.0, 2147)

In [186]:
success = power_on(robot)

FaultedError: bosdyn.api.PowerCommandResponse (FaultedError): Cannot power on due to a fault; inspect FaultState for more info.

In [ ]:
bdcrc.blocking_selfright(command_client, timeout_sec = 20)

In [ ]:
bdcrc.blocking_stand(command_client, timeout_sec = 20)

In [166]:
blocking_stow(command_client)

response = upload_graph_and_snapshots(graph_nav_client, upload_filepath)
success = graph_localize_fiducial(command_client, graph_nav_client, robot_state_client, robot)
waypoint_list = create_waypoint_list(graph_nav_client)


In [176]:
nav_to_cmd_id = graph_nav_client.navigate_to(waypoint_list[0], cmd_duration = 15)
wait_until_finished_nav(graph_nav_client, nav_to_cmd_id)

In [177]:
move_robot_relative(command_client, robot, 1, -0.2, np.pi/2)

In [178]:
prev_graph_state = graph_del_save_state(graph_nav_client, robot)

In [179]:
move_robot_relative(command_client, robot, 0.4, -2, np.pi/3)

In [174]:
response = upload_graph_and_snapshots(graph_nav_client, upload_filepath)
response = graph_localize_saved_state(graph_nav_client, robot, prev_graph_state)

In [126]:
prev_frame_tree_snapshot = robot.get_frame_tree_snapshot()
#prev_vision_wrt_body_se3 = get_a_tform_b(prev_frame_tree_snapshot, frame_a = 'body', frame_b = 'vision')

prev_loc_response = graph_nav_client.get_localization_state()
prev_waypoint_tform_body_proto = prev_loc_response.localization.waypoint_tform_body
prev_waypoint_tform_body_se3 = bosdyn.client.math_helpers.SE3Pose.from_proto(prev_waypoint_tform_body_proto)
prev_body_tform_waypoint_proto = prev_waypoint_tform_body_se3.inverse().to_proto()
prev_waypoint_id = prev_loc_response.localization.waypoint_id

prev_fts_with_waypoint = real_frame_add(prev_frame_tree_snapshot, 
                                    prev_body_tform_waypoint_proto,
                                    'body',
                                    'waypoint')

vision_tform_waypoint = get_a_tform_b(prev_fts_with_waypoint, 'vision', 'waypoint')

In [135]:
vision_tform_waypoint

In [127]:
response = graph_nav_client.clear_graph()

In [128]:
move_robot_relative(command_client, robot, 0, -0.5, np.pi/3)

In [129]:
frame_tree_snapshot = robot.get_frame_tree_snapshot()

fts_with_waypoint = real_frame_add(frame_tree_snapshot, 
                                   vision_tform_waypoint.to_proto(),
                                   'vision',
                                   'waypoint')

waypoint_tform_body_proto = get_a_tform_b(fts_with_waypoint, 'waypoint', 'body').to_proto()

In [130]:
fid_init_code_NO_FID = graph_nav_pb2.SetLocalizationRequest.FiducialInit.FIDUCIAL_INIT_NO_FIDUCIAL

init_guess_loc = nav_pb2.Localization(waypoint_id = prev_waypoint_id,
                                      waypoint_tform_body = waypoint_tform_body_proto)

response = upload_graph_and_snapshots(graph_nav_client, upload_filepath)

response = graph_nav_client.set_localization(initial_guess_localization=init_guess_loc,
                              fiducial_init = fid_init_code_NO_FID)

In [131]:
response

waypoint_id: "mono-eagle-pUcEzJeZxsXlz.HlH7T7yA=="
waypoint_tform_body {
  position {
    x: 0.3118391676124863
    y: -0.3239014827730545
    z: -0.01856264091242215
  }
  rotation {
    x: 0.010777151399914578
    y: -0.007631501074881512
    z: -0.8535470346646697
    w: -0.5208483631592995
  }
}

In [133]:
waypoint_list[9]

'mono-eagle-pUcEzJeZxsXlz.HlH7T7yA=='

In [134]:
nav_to_cmd_id = graph_nav_client.navigate_to(waypoint_list[8], cmd_duration = 15)
wait_until_finished_nav(graph_nav_client, nav_to_cmd_id)

In [24]:
current_graph = graph_nav_client.download_graph()

In [29]:
list(current_graph.DESCRIPTOR.fields_by_name)

['waypoints', 'edges', 'anchoring']

In [37]:
prev_loc_response.robot_kinematics.transforms_snapshot

child_to_parent_edge_map {
  key: "body"
  value {
    parent_tform_child {
      position {
      }
      rotation {
        w: 1.0
      }
    }
  }
}
child_to_parent_edge_map {
  key: "flat_body"
  value {
    parent_frame_name: "body"
    parent_tform_child {
      position {
      }
      rotation {
        x: 0.0035347850061953068
        y: 0.0005604380858130753
        z: 1.9257468011346646e-06
        w: 0.999993622303009
      }
    }
  }
}
child_to_parent_edge_map {
  key: "gpe"
  value {
    parent_frame_name: "odom"
    parent_tform_child {
      position {
        x: 1.5602593421936035
        y: 0.9091336727142334
        z: -0.23126977682113647
      }
      rotation {
        x: 0.001912608160637319
        y: 0.0013491030549630523
        z: 2.5803124117373955e-06
        w: 0.9999972581863403
      }
    }
  }
}
child_to_parent_edge_map {
  key: "hand"
  value {
    parent_frame_name: "body"
    parent_tform_child {
      position {
        x: 0.5540702939033508
    

In [140]:
current_graph.anchoring

anchors {
  id: "apodal-aphid-sziFphCcuV+rd8cpxqEbkw=="
  seed_tform_waypoint {
    position {
      x: 1.5760424479846835
      y: -3.1721811880210447
      z: 0.34087623689057606
    }
    rotation {
      x: 0.00819316783004938
      y: 0.0071058028037067535
      z: -0.6568205968961189
      w: 0.7539688432583809
    }
  }
}
anchors {
  id: "averse-camel-cjfcW+r8KsgZy36RB3R51A=="
  seed_tform_waypoint {
    position {
      x: -1.6197538828906595
      y: -3.2474428231663324
      z: 0.32528016760392964
    }
    rotation {
      x: 0.0003728482478630516
      y: 0.0031400851750743946
      z: 0.697322581403387
      w: 0.7167504467196377
    }
  }
}
anchors {
  id: "banned-dayfly-PJb.wb5lyGb9AAv9M7bcbg=="
  seed_tform_waypoint {
    position {
      x: -1.2418929326322292
      y: 1.2314549356932
      z: 0.3680433686765155
    }
    rotation {
      x: -0.000798685536128824
      y: 0.005753985058187916
      z: 0.021868743592423592
      w: 0.9997439731306479
    }
  }
}
anchors

In [13]:
prev_loc_response = graph_nav_client.get_localization_state()
prev_graph_loc_proto = prev_loc_response.localization

prev_waypoint_id = prev_graph_loc_proto.waypoint_id

prev_body_wrt_waypoint_proto = prev_graph_loc_proto.waypoint_tform_body
prev_body_wrt_waypoint_se3 = bosdyn.client.math_helpers.SE3Pose.from_proto(
    prev_body_wrt_waypoint_proto)

prev_frame_tree_snapshot = robot.get_frame_tree_snapshot()
prev_vision_wrt_body_se3 = get_a_tform_b(prev_frame_tree_snapshot, frame_a = 'body', frame_b = 'vision')

vision_wrt_waypoint_se3 = (prev_vision_wrt_body_se3 * prev_body_wrt_waypoint_se3)
# prev_graph_state = {'prev_waypoint_id':prev_waypoint_id, 
#                     'vision_to_waypoint_se3': vision_to_waypoint_se3,
#                     'prev_body_wrt_waypoint_se3': prev_body_wrt_waypoint_se3}

In [14]:
vision_wrt_waypoint_se3 = prev_vision_wrt_body_se3 * prev_body_wrt_waypoint_se3

In [15]:
move_robot_relative(command_client, robot, 0.8,0,np.pi/2)

In [257]:
# move_robot_relative(command_client, robot,0,0,232*np.pi/180)

In [16]:
frame_tree_snapshot = robot.get_frame_tree_snapshot()
body_wrt_vision_se3 = get_a_tform_b(frame_tree_snapshot, frame_a = 'vision', frame_b = 'body')

In [18]:
prev_body_wrt_waypoint_proto

position {
  x: 0.6564040848204789
  y: -0.05674467650074774
  z: 0.006177651296188025
}
rotation {
  x: 0.00287228816064919
  y: -0.003603355356269125
  z: -0.00805602304170006
  w: 0.9999559518467156
}

In [26]:
body_wrt_waypoint_se3.inverse().to_proto()

position {
  x: -0.49830392362308473
  y: 2.014986948942526
  z: 0.004389346150220408
}
rotation {
  x: 0.00580432017521404
  y: -0.0029056665021753953
  z: 0.7000428433727102
  w: 0.7140701398165175
}

In [21]:
body_wrt_vision_se3.to_proto()

position {
  x: -0.8931301297386003
  y: 0.9557345602139401
  z: 0.3029740269415822
}
rotation {
  x: -0.0011382473167032003
  y: -0.0011360186617821455
  z: 0.9312471151351929
  w: -0.36438482999801636
}

In [250]:
response = upload_graph_and_snapshots(graph_nav_client, upload_filepath)

In [22]:
fid_init_code_NO_FID = graph_nav_pb2.SetLocalizationRequest.FiducialInit.FIDUCIAL_INIT_NO_FIDUCIAL

# frame_tree_snapshot = robot.get_frame_tree_snapshot()
# #get_a_tform_b returns transformation such that a_coords = tform * b_coords
# body_to_vision_se3 = get_a_tform_b(frame_tree_snapshot, frame_a = 'vision', frame_b = 'body')
# body_wrt_waypoint_proto = (body_to_vision_se3 * vision_to_waypoint_se3).to_proto()

body_wrt_waypoint_se3 = (body_wrt_vision_se3 * vision_wrt_waypoint_se3).inverse()
body_wrt_waypoint_proto = body_wrt_waypoint_se3.to_proto()


# init_guess_loc = nav_pb2.Localization(waypoint_id = prev_waypoint_id,
#                                   waypoint_tform_body = body_wrt_waypoint_proto)

# response = graph_nav_client.set_localization(initial_guess_localization=init_guess_loc,
#                               fiducial_init = fid_init_code_NO_FID)

In [260]:
body_wrt_waypoint_proto

position {
  x: -1.5529113437677817
  y: -0.10633248635573772
  z: -0.0030849759646797005
}
rotation {
  x: -0.0016134210476990768
  y: 0.0009332780906984008
  z: -0.27925347152924895
  w: 0.9602157939731498
}

In [261]:
quat = bosdyn.client.math_helpers.Quat.from_proto(body_wrt_waypoint_proto.rotation)
angles = np.array([quat.to_yaw(), quat.to_pitch(), quat.to_roll()])
print('z','y', 'x')
print(np.round(angles * 180/np.pi,2))

z y x
[-32.43   0.05  -0.21]


In [92]:
response = upload_graph_and_snapshots(graph_nav_client, upload_filepath)
#response = graph_localize_saved_state(graph_nav_client, robot, prev_graph_state)

In [94]:
response = graph_localize_saved_state(graph_nav_client, robot, prev_graph_state)

InvalidRequestError: bosdyn.api.graph_nav.SetLocalizationResponse (InvalidRequestError): to_waypoint not set?

In [43]:
def rand_imgs_thru_det(command_client, image_client, gl, det, look_at, num_images = 4):
    for i in range(num_images):
        move_to = [np.random.uniform(0.3,1),
               np.random.uniform(-0.1,0.1),
               np.random.uniform(0.4,0.8)]

        blocking_stow(command_client)

        move_to_and_look_at(command_client, move_to, look_at)
        
        time.sleep(0.5)
        
        hand_color_image, _ = cap_hand_image(image_client, "hand_color_image")
        
        image_query = mat_thru_det(gl, det, hand_color_image)

In [44]:
for wp, move, look_at in all_wp_move_look:
    print(wp, move, look_at)
    nav_to_cmd_id = graph_nav_client.navigate_to(waypoint_list[wp], cmd_duration = 15)
    wait_until_finished_nav(graph_nav_client, nav_to_cmd_id)
    move_robot_relative(command_client, robot, move[0], move[1], move[2])
    
    prev_graph_state = graph_del_save_state(graph_nav_client, robot)
    
    #move_to_and_look_at(command_client, [0.8, 0, 0.6], look_at)
    gripper_open(command_client)
    rand_imgs_thru_det(command_client, image_client, gl, light_det, look_at)
    blocking_stow(command_client)
    
    response = upload_graph_and_snapshots(graph_nav_client, upload_filepath)
    response = graph_localize_saved_state(graph_nav_client, robot, prev_graph_state)

10 [0, 0, 0.7853981633974483] [2, 0, 2.5]


18 [0, 0, 1.5707963267948966] [1, 0, 5.0]


KeyboardInterrupt: 

In [47]:
blocking_stow(command_client)

True

In [15]:
# [2,0,2.5] for 10

# [1,0,5] for 18

# [2, 0, 4] for 22

# [3, 0, 3] for 0




In [70]:
nav_to_cmd_id = graph_nav_client.navigate_to(waypoint_list[0], cmd_duration = 15)
wait_until_finished_nav(graph_nav_client, nav_to_cmd_id)
move_robot_relative(command_client, robot, 0, 0, np.pi/4)

KeyboardInterrupt: 

In [56]:
move_to_and_look_at(command_client, [0.8,0,0.5], [3, 0, 3])

In [72]:
move_robot_relative(command_client, robot, 0, -2, 0)

In [ ]:
wp_ind_list = [0, 8, 10, 12, 15]
for wp_ind in wp_ind_list:
    blocking_stow(command_client)
    nav_to_cmd_id = graph_nav_client.navigate_to(waypoint_list[wp_ind], cmd_duration = 15)
    wait_until_finished_nav(graph_nav_client, nav_to_cmd_id)
    prev_graph_state = graph_del_save_state(graph_nav_client, robot)
    gripper_open(command_client)
    rand_imgs_thru_det(command_client, image_client, gl, rubiks_det)
    gripper_close(command_client)
    upload_graph_and_snapshots(graph_nav_client, upload_filepath)
    response = graph_localize_saved_state(graph_nav_client, robot, prev_graph_state)